<a href="https://colab.research.google.com/github/vishnu-chand/faceFocusBot/blob/main/faceFocusBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%load_ext autoreload
%autoreload 2

import os
if '' or not os.path.exists('/content/pixUtils'):
    !git clone https://github.com/vishnu-chand/pixUtils.git;mv pixUtils deleteMe;mv deleteMe/pixUtils .;rm -rf deleteMe
    !pip install youtube_dl

import dlib
from pixUtils import *

def dbbox2bbox(img, dbbox):
    x = dbbox.left()
    y = dbbox.top()
    w = dbbox.right() - x
    h = dbbox.bottom() - y
    return frameFit(img, (x, y, w, h))


def bboxScale(img, bbox, roi):
    bbox = bbox * roi  # roi = top, left, width, height
    return frameFit(img, bbox)


def disp(bbox, stableBbox, faceBox, img):
    stableImg = getSubImg(img.copy(), stableBbox)
    stableImg = bboxLabel(stableImg, "output")
    (stH, stW), thickness = stableImg.shape[:2], 15
    stableImg = bboxLabel(stableImg, "", [0, 0, stW - thickness / 2, stH - thickness / 2], (0, 128, 0), thickness=thickness)
    img = bboxLabel(img, "", faceBox)
    simg = getSubImg(img, bbox).copy()
    simg = bboxLabel(simg, "with jitters")
    img = bboxLabel(img, "roi", stableBbox, (0, 255, 255))
    simg = photoframe([stableImg, simg])
    (sH, sW), (h, stH) = simg.shape[:2], img.shape[:2]
    img = imResize(img, [sH, int(sH / sW * stH)])
    debug = cv2.hconcat([simg, img])
    return debug


def main(vPath, startSec, stopSec, roi, desPath):
    detector = dlib.get_frontal_face_detector()
    stableBbox, simg, faceBox, cam = None, None, None, cv2.VideoCapture(vPath)
    with VideoWriter(desPath, cam) as vWriter:
        for fno, ftm, img in videoPlayer(cam, startSec=startSec, stopSec=stopSec):
            bbox = detector(img, 0)
            if bbox:
                faceBox = dbbox2bbox(img, bbox[0])
                bbox = bboxScale(img, faceBox, roi)
                if stableBbox is None:
                    stableBbox = bbox
                stableBbox = stableBbox * .95 + bbox * .05
            else:
                bbox = stableBbox
            debug = disp(bbox, stableBbox, faceBox, img)
            vWriter.write(debug)
            # key = showImg('faceStable_resized', debug, 1)
            # if key == 27:
                # break

downloadDB('youtube+https://www.youtube.com/watch?v=Lf3kPo8WYM4', '/content/db')
vPath = rglob('/content/db/*.mp4')[0]
main(vPath, startSec=25, stopSec=57, roi=np.array([.5, .7, 8, 10]), desPath='/content/out.mp4')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
